<a href="https://colab.research.google.com/github/jarvisx17/Langchain-Chatbots/blob/main/falcon_%2B_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install -q accelerate einops
!pip install -q langchain
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.5 MB/s eta 0:00:00


In [ ]:
from torch import cuda, bfloat16
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

#creating a model
fmodel = AutoModelForCausalLM.from_pretrained(
    'tiiuae/falcon-7b-instruct',
    trust_remote_code=True,
    torch_dtype=bfloat16
).to(device)
fmodel.eval()
fmodel.to(device)
print(f'Model loaded on {device}')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b-instruct')

In [ ]:
gen_text = transformers.pipeline(
    model=fmodel,
    tokenizer=tokenizer,
    task='text-generation',
    return_full_text=True,
    device=device,
    max_length=10000,
    temperature=0.1,
    top_p=0.15, #select from top tokens whose probability adds up to 15%
    top_k=0, #selecting from top 0 tokens
    repetition_penalty=1.1, #without a penalty, output starts to repeat
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
result = gen_text("What is the name of the first president of the united arab emirates?")
print(result[0]['generated_text'])


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory,ConversationSummaryMemory
from langchain.llms import HuggingFacePipeline

#creating a new conversational template
template = """You are an informative assistant chatting with a human.
{chat_history}
Human:{human_input}
Assistant:"""
#creating the prompt
prompt = PromptTemplate(
    input_variables=["chat_history","human_input"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm = HuggingFacePipeline(pipeline=gen_text)
#adding memory to the llm chain
llm_chain = LLMChain(llm=llm,prompt=prompt,memory=memory)


In [ ]:

#1st prompt
llm_chain.predict(human_input='What is the first president of UAE?')

In [ ]:

#2nd prompt
llm_chain.predict(human_input='Who was the president that came after him?')